In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

### Load cleaned comments and article data 

In [3]:
%%time

# Load cleaned comment data
comments = pd.read_csv('../data/cleaned/comments.csv', header=0, parse_dates=['scrape_datetime','comment_datetime_clean'])
comments.shape

Wall time: 1min 31s


(5689832, 5)

In [3]:
comments.head()

,article_url,commentor,comment_body,scrape_datetime,comment_datetime_clean
0,https://profootballtalk.nbcsports.com/2020/10/...,amaf21,\nthe best bet is the cowboys giving up 10 sac...,2020-10-28 19:15:17.148123,2020-10-25 10:57:00
1,https://profootballtalk.nbcsports.com/2020/10/...,amaf21,"\n1 strip sack for a safety down, 9 more to go...",2020-10-28 19:15:17.148123,2020-10-25 13:20:00
2,https://profootballtalk.nbcsports.com/2020/10/...,eagleswin,\nTalent still trumps all in the NFL. In betw...,2020-10-28 19:15:17.169123,2020-10-25 10:46:00
3,https://profootballtalk.nbcsports.com/2020/10/...,cobrala2,\nTom Brady wants this guy.\n,2020-10-28 19:15:17.169123,2020-10-25 10:54:00
4,https://profootballtalk.nbcsports.com/2020/10/...,freakylj8,\nUse AB to let Godwin get healthy I guess\n,2020-10-28 19:15:17.169123,2020-10-25 11:01:00


In [4]:
%%time

# Load cleaned article data
articles = pd.read_csv('../data/cleaned/articles.csv', header=0, parse_dates=['scrape_datetime','post_datetime'])
articles.shape

Wall time: 7.56 s


(202862, 8)

In [ ]:
articles.head()

### Engineer numerical features

In [21]:
# Calculate the total number of comments by each commentor
commentor_features = pd.DataFrame(comments.groupby(['commentor']).size())
commentor_features['commentor'] = commentor_features.index
commentor_features.columns = ['total_number_of_comments','commentor']
commentor_features.to_csv('../data/cluster_features/01_total_number_of_comments.csv', header=True, index=True)
commentor_features.head()

,total_number_of_comments,commentor
commentor,,
\nguitarkevin,1,\nguitarkevin
\npraetorian12,1,\npraetorian12
"""All Eyez On Me"" in theaters NOW!!!",6,"""All Eyez On Me"" in theaters NOW!!!"
"""All Eyez On Me"" in theaters june 16 2017",49,"""All Eyez On Me"" in theaters june 16 2017"
"""Coach""Davis",31,"""Coach""Davis"


In [22]:
# Calculate the number of unique articles the commentor commented on
unique_articles = comments.groupby(['commentor','article_url']).size()
unique_articles_df = pd.DataFrame(unique_articles.groupby(['commentor']).size())
unique_articles_df.columns = ['number_of_articles_commented_on']
unique_articles_df.to_csv('../data/cluster_features/02_unique_articles_df.csv', header=True, index=True)
unique_articles_df.head()

,number_of_articles_commented_on
commentor,
\nguitarkevin,1
\npraetorian12,1
"""All Eyez On Me"" in theaters NOW!!!",6
"""All Eyez On Me"" in theaters june 16 2017",49
"""Coach""Davis",30


In [23]:
# Calculate the number of unique articles the commentor commented on exactly once
unique_articles_single_comment = comments.groupby(['commentor','article_url']).size()
unique_articles_single_comment_df = pd.DataFrame(unique_articles_single_comment)
unique_articles_single_comment_df = unique_articles_single_comment_df[unique_articles_single_comment_df[0] == 1]
unique_articles_single_comment_df = pd.DataFrame(unique_articles_single_comment_df.groupby(['commentor']).size())
unique_articles_single_comment_df.columns = ['number_of_articles_w_exactly_one_comment']
unique_articles_single_comment_df.to_csv('../data/cluster_features/03_unique_articles_single_comment_df.csv', header=True, index=True)
# unique_articles_single_comment_df.head()

In [25]:
# Calculate the number of unique articles the commentor commented on more than once
unique_articles_mulitple_comment = comments.groupby(['commentor','article_url']).size()
unique_articles_mulitple_comment_df = pd.DataFrame(unique_articles_mulitple_comment)
unique_articles_mulitple_comment_df = unique_articles_mulitple_comment_df[unique_articles_mulitple_comment_df[0] > 1]
unique_articles_mulitple_comment_df = pd.DataFrame(unique_articles_mulitple_comment_df.groupby(['commentor']).size())
unique_articles_mulitple_comment_df.columns = ['number_of_articles_w_more_than_one_comment']
unique_articles_mulitple_comment_df.to_csv('../data/cluster_features/04_unique_articles_mulitple_comment_df.csv', header=True, index=True)
# unique_articles_mulitple_comment_df.head()

In [26]:
# Calculate how long (in days) a commentor has been active on pft
commentor_activity_duration = comments.groupby(['commentor']).agg({'comment_datetime_clean':['min','max']})
commentor_activity_duration.columns = commentor_activity_duration.columns.droplevel()
commentor_activity_duration['commentor_activity_duration_in_days'] = (commentor_activity_duration['max'] - commentor_activity_duration['min']).dt.days
commentor_activity_duration.to_csv('../data/cluster_features/05_commentor_activity_duration.csv', header=True, index=True)
# commentor_activity_duration.head()

In [27]:
# Calcualte the length of the commentor's username
commentor_username_length = comments.groupby(['commentor']).size()
commentor_username_length = pd.DataFrame(commentor_username_length)
commentor_username_length['username'] = commentor_username_length.index
commentor_username_length['username_length'] = commentor_username_length['username'].str.len()
commentor_username_length.drop([0], axis=1, inplace=True)

# Calculate the number of letters, numbers, and spaces in the commentor's username
commentor_username_length['username_alpha_chars'] = commentor_username_length['username'].apply(lambda username: sum(x.isalpha() for x in username))
commentor_username_length['username_numeric_chars'] = commentor_username_length['username'].apply(lambda username: sum(x.isdigit() for x in username))
commentor_username_length['username_space_chars'] = commentor_username_length['username'].apply(lambda username: sum(x.isspace() for x in username))
commentor_username_length.to_csv('../data/cluster_features/06_commentor_username_length.csv', header=True, index=True)
# commentor_username_length.head()

In [28]:
# Calculate the mean, median, min and max length of comments (characters)
comments['comment_body_length'] = comments['comment_body'].str.len()
commentor_comment_body_metrics = comments.groupby(['commentor']).agg({'comment_body_length':['mean','median','min','max']})
commentor_comment_body_metrics.columns = commentor_comment_body_metrics.columns.droplevel()
commentor_comment_body_metrics.columns = ['comment_length_mean','comment_length_median','comment_length_min','comment_length_max']
commentor_comment_body_metrics.to_csv('../data/cluster_features/07_commentor_comment_body_metrics.csv', header=True, index=True)
# commentor_comment_body_metrics.head()

In [29]:
# Calculate the average, median, min, max hours between when article was published and comment was made
articles_w_dates = articles.drop_duplicates(subset=['article_url','post_datetime'])
comments_between = pd.merge(comments, articles_w_dates[['article_url','post_datetime']], how='left', on='article_url')
comments_between = comments_between[(comments_between.comment_datetime_clean >= comments_between.post_datetime)]
comments_between['hours_btween'] = (comments_between.comment_datetime_clean - comments_between.post_datetime) / pd.Timedelta(hours=1)
 
hours_between_metrics = comments_between.groupby(['commentor']).agg({'hours_btween':['mean','median','min','max']})
hours_between_metrics.columns = hours_between_metrics.columns.droplevel()
hours_between_metrics.columns = ['hours_between_mean','hours_between_median','hours_between_min','hours_between_max']
hours_between_metrics.to_csv('../data/cluster_features/08_hours_between_metrics.csv', header=True, index=True)
# hours_between_metrics.head()

In [30]:
# Calculate which days of the week comments were made on
comments['comment_date_dow'] = comments['comment_datetime_clean'].dt.day_name()
comments_dow = pd.pivot_table(comments[['article_url','commentor','comment_date_dow']], index=['commentor'],
                    columns=['comment_date_dow'], aggfunc='count', fill_value=0)
comments_dow.columns = comments_dow.columns.droplevel()
comments_dow.columns = ['comments_on_' + c for c in comments_dow.columns]
comments_dow.to_csv('../data/cluster_features/09_comments_dow.csv', header=True, index=True)
# comments_dow.head()

In [31]:
# Calculate which hours of the day comments were made on
comments['comment_date_hour'] = comments['comment_datetime_clean'].dt.hour
comments_hour = pd.pivot_table(comments[['article_url','commentor','comment_date_hour']], index=['commentor'],
                    columns=['comment_date_hour'], aggfunc='count', fill_value=0)
comments_hour.columns = comments_hour.columns.droplevel()
comments_hour.columns = ['comments_on_hour_' + str(c) for c in comments_hour.columns]
comments_hour.to_csv('../data/cluster_features/10_comments_hour.csv', header=True, index=True)
# comments_hour.head()

,comments_on_hour_0,comments_on_hour_1,comments_on_hour_2,comments_on_hour_3,comments_on_hour_4,comments_on_hour_5,comments_on_hour_6,comments_on_hour_7,comments_on_hour_8,comments_on_hour_9,...,comments_on_hour_14,comments_on_hour_15,comments_on_hour_16,comments_on_hour_17,comments_on_hour_18,comments_on_hour_19,comments_on_hour_20,comments_on_hour_21,comments_on_hour_22,comments_on_hour_23
commentor,,,,,,,,,,,,,,,,,,,,,
\nguitarkevin,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
\npraetorian12,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
"""All Eyez On Me"" in theaters NOW!!!",0,0,0,0,0,0,0,2,0,0,...,0,0,1,1,0,1,0,0,0,0
"""All Eyez On Me"" in theaters june 16 2017",0,0,0,0,0,0,2,9,3,2,...,2,2,2,5,1,3,3,0,1,1
"""Coach""Davis",1,0,0,0,0,0,0,0,0,0,...,2,0,1,3,3,3,4,2,2,0


In [32]:
# Count number of comment made "in-season" vs "off-season"
# In-season being between 9/1 and 2/1, inclusive
comments['comment_date_month'] = comments['comment_datetime_clean'].dt.month
comments['in_season_flag'] = np.where((comments['comment_date_month'] >= 9) | (comments['comment_date_month'] <= 2), 1, 0)
in_season_comments = comments.groupby(['commentor']).agg({'in_season_flag':['count','sum']})
in_season_comments.columns = in_season_comments.columns.droplevel()
in_season_comments.columns = ['total_comments','number_in_season_comments']
in_season_comments['number_out_season_comments'] = in_season_comments['total_comments'] - in_season_comments['number_in_season_comments']
in_season_comments.to_csv('../data/cluster_features/11_in_season_comments.csv', header=True, index=True)
# in_season_comments.head()

In [6]:
# Calculate the most number of comments each commentor posted in a single day
commentor_max_comments = pd.DataFrame(comments.groupby(['commentor','comment_datetime_clean']).size()).reset_index()
commentor_max_comments_df = pd.DataFrame(commentor_max_comments.groupby(['commentor'])[0].max())
commentor_max_comments_df.columns = ['max_number_comments_in_single_day']
commentor_max_comments_df.to_csv('../data/cluster_features/12_commentor_max_comments_df.csv', header=True, index=True)
# commentor_max_comments_df.head()

,max_number_comments_in_single_day
commentor,
\nguitarkevin,1
\npraetorian12,1
"""All Eyez On Me"" in theaters NOW!!!",1
"""All Eyez On Me"" in theaters june 16 2017",1
"""Coach""Davis",1


### Engineer categorical features